In [2]:
#https://velog.io/@huttzza/%EC%8B%A4%EC%8B%9C%EA%B0%84-%EC%96%BC%EA%B5%B4-%EC%9D%B8%EC%8B%9D-%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8-1%EC%B0%A8-%EA%B5%AC%ED%98%84
#카메라를 켜서 사진을 찍고 지정해둔 폴더에 저장

import cv2 #OpenCV 영상처리

#classifier
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#video caputure setting
capture = cv2.VideoCapture(1) # initialize, # is camera number
capture.set(cv2.CAP_PROP_FRAME_WIDTH,1280) #CAP_PROP_FRAME_WIDTH == 3
capture.set(cv2.CAP_PROP_FRAME_HEIGHT,720) #CAP_PROP_FRAME_HEIGHT == 4

#console message
face_id = input('\n enter user id end press <return> ==> ')
print("\n [INFO] Initializing face capture. Look the camera and wait ...")

count = 0 # # of caputre face images
#영상 처리 및 출력
while True: 
    ret, frame = capture.read() #카메라 상태 및 프레임
    # cf. frame = cv2.flip(frame, -1) 상하반전 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #흑백으로
    faces = faceCascade.detectMultiScale(
        gray,#검출하고자 하는 원본이미지
        scaleFactor = 1.2, #검색 윈도우 확대 비율, 1보다 커야 한다
        minNeighbors = 6, #얼굴 사이 최소 간격(픽셀)
        minSize=(20,20) #얼굴 최소 크기. 이것보다 작으면 무시
    )

    #얼굴에 대해 rectangle 출력
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        #inputOutputArray, point1 , 2, colorBGR, thickness)
        count += 1
        cv2.imwrite("/Users/gangseoggyu/Desktop/opencv/image_1/image"+str(face_id)+'.'+str(count)+".jpg",gray[y:y+h, x:x+w])
    cv2.imshow('image',frame)

    #종료 조건
    if cv2.waitKey(1) > 0 : break #키 입력이 있을 때 반복문 종료
    elif count >= 100 : break #100 face sample
 

capture.release() #메모리 해제
cv2.destroyAllWindows()#모든 윈도우 창 닫기



 enter user id end press <return> ==>  



 [INFO] Initializing face capture. Look the camera and wait ...


In [5]:
# 저장된 이미지들을 가지고 학습하는 코드<원래 코드>
import cv2
import numpy as np #배열 계산 용이
from PIL import Image #python imaging library
import os

detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

recognizer = cv2.face.LBPHFaceRecognizer_create()  

path = '/Users/gangseoggyu/Desktop/opencv/image_1'

def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    #listdir : 해당 디렉토리 내 파일 리스트
    #path + file Name : 경로 list 만들기
    
    faceSamples = []
    ids = []
    for imagePath in imagePaths: #각 파일마다
        #흑백 변환
        PIL_img = Image.open(imagePath).convert('L') #L : 8 bit pixel, bw
        img_numpy = np.array(PIL_img, 'uint8')

        #user id
        id = int(os.path.split(imagePath)[-1].split(".")[1])#마지막 index : -1

        #학습을 위한 얼굴 샘플
        faces = detector.detectMultiScale(img_numpy)
        
        for(x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)

    return faceSamples, ids
    
print('\n [INFO] Training faces. It will take a few seconds. Wait ...')
faces, ids = getImagesAndLabels(path)

recognizer.train(faces,np.array(ids)) #학습

recognizer.write('/Users/gangseoggyu/Desktop/opencv/trainer.yml')
print('\n [INFO] {0} faces trained. Exiting Program'.format(len(np.unique(ids))))


 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 100 faces trained. Exiting Program


In [4]:
# 저장된 이미지들을 가지고 학습하는 코드<PyTorch 사용 코드>
import cv2
import torch
import numpy as np #배열 계산 용이
from PIL import Image #python imaging library
import os

detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

recognizer = cv2.face.LBPHFaceRecognizer_create()  

path = '/Users/gangseoggyu/Desktop/opencv/image_1'

def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    #listdir : 해당 디렉토리 내 파일 리스트
    #path + file Name : 경로 list 만들기
    
    faceSamples = []
    ids = []
    for imagePath in imagePaths: #각 파일마다
        #흑백 변환
        cv_img = cv2.imread(imagePath)
        resized_image = cv2.resize(cv_img, (224, 224))
        tensor_image = torch.from_numpy(resized_image)

        # img_numpy = np.array(PIL_img, 'uint8')

        # 얼굴 특징 추출
        model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
        features = model.forward(tensor_image)

        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        for epoch in range(100):#밑으로 학습 코드 작성
        
        #user id
        id = int(os.path.split(imagePath)[-1].split(".")[1])#마지막 index : -1

        #학습을 위한 얼굴 샘플
        faces = detector.detectMultiScale(tensor_image)
        
        for(x,y,w,h) in faces:
            faceSamples.append(tensor_image[y:y+h,x:x+w])
            ids.append(id)

    return faceSamples, ids
    
print('\n [INFO] Training faces. It will take a few seconds. Wait ...')
faces, ids = getImagesAndLabels(path)

recognizer.train(faces,np.array(ids)) #학습

recognizer.write('/Users/gangseoggyu/Desktop/opencv/trainer.yml')
print('\n [INFO] {0} faces trained. Exiting Program'.format(len(np.unique(ids))))


 [INFO] Training faces. It will take a few seconds. Wait ...


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'detectMultiScale'
> Overload resolution failed:
>  - image is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'image'


In [6]:
# 학습한 결과를 가지고 누구인지 알려주는 코드
import cv2
import numpy as np

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('/Users/gangseoggyu/Desktop/opencv/trainer.yml')
cascadePath = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascadePath)

font = cv2.FONT_HERSHEY_SIMPLEX

id = 0

names = ['None','sumin','dongjun','minji']

cam = cv2.VideoCapture(1)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1980)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

minW = 0.1 * cam.get(cv2.CAP_PROP_FRAME_WIDTH)
minH = 0.1 * cam.get(cv2.CAP_PROP_FRAME_HEIGHT)

while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=6,
        minSize=(int(minW), int(minH))
    )

    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0),2)
        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])

        if confidence > 55 :
            id = names[id]
        else:
            id = "unknown"
        
        confidence = "  {0}%".format(round(100-confidence))

        cv2.putText(img,str(id), (x+5,y-5),font,1,(255,255,255),2)
        cv2.putText(img,str(confidence), (x+5,y+h-5),font,1,(255,255,0),1)
    
    cv2.imshow('camera',img)
    if cv2.waitKey(1) > 0 : break

print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

IndexError: list index out of range

In [2]:
conda install pytorch

Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
